# PPO (Proximal Policy Optimization)

## 前言：為什麼我們需要 PPO？

### DPO 的侷限性
**情境 1: 動態任務分布：**

問題：你在訓練一個遊戲 AI，遊戲規則會隨著玩家等級改變

DPO：需要預先收集所有場景的偏好資料（不可行）

PPO：可以邊玩邊學，動態適應


**情境 2: 探索式學習：**

問題：你希望模型自己發現更好的解法（如程式碼改善）

DPO：只能學習已知的好壞範例

PPO：可以透過 reward 引導探索

**情境 3: 複雜的多步驟推理：**

問題：數學證明，每一步都有分數，最後才知道對錯

DPO：難以處理中間步驟的信用分配（credit assignment）

PPO：可以透過 reward shaping 逐步引導

**情境 4: 需要持續適應：**

問題：使用者偏好會隨時間改變

DPO：需要重新收集偏好資料並重訓

PPO：可以用新的 reward signal 持續微調

**核心差異：**
* DPO 是離線學習：從固定的偏好資料學習
* PPO 是線上學習：從與環境互動中學習

這不是說 PPO 更好，而是它們適用於不同的場景。如果你的任務符合上述情境，那麼理解 PPO 就是必要的。

## RLHF 架構總覽

### 三個關鍵模型
1. Policy Model (π_θ)

    └─ 我們要訓練的模型

    └─ 輸入：prompt   

    └─ 輸出：response  

2. Reward Model (RM)   

    └─ 評分器，告訴我們回答有多好  

    └─ 輸入：(prompt, response)   

    └─ 輸出：scalar reward    

3. Reference Model (π_ref)  

    └─ 參考模型，防止 policy 偏離太遠  

    └─ 通常是 SFT 後的模型 

    └─ 參數凍結，不訓練   

## 訓練目標

**PPO 的目標是：**

In [ ]:
最大化 expected reward - β × KL_divergence(π_θ || π_ref)

# 「獲得高分」但「不要跟原本的模型差太多」

1. Exploitation：最大化 reward → 找到高分的回答
2. Constraint：KL penalty → 不要偏離太遠

## Reward Model：為什麼需要它？

在 DPO 中，我們直接用偏好資料訓練，不需要 reward model。為什麼 PPO 需要？

### Reward Model 的角色

功能： 將「人類的偏好」蒸餾成一個自動評分器

In [ ]:
# Reward Model 的輸入輸出
input: (prompt, response)
output: scalar score  # 越高越好

# 範例
reward("如何學 Python?", "建議從官方教學開始...") = 0.8
reward("如何學 Python?", "不知道") = -0.3

### 為什麼需要 Reward Model？

因為 PPO 是 **線上學習**：

PPO 的訓練迴圈：
1. Policy 生成新的 response
2. Reward Model 評分
3. 用評分更新 policy
4. 重複...

這個過程中，模型會不斷生成訓練資料中沒見過的回答。我們需要一個自動評分器來告訴 policy「這個新回答好不好」。

### Reward Model 的訓練

Reward Model 本身是用 偏好資料 訓練的：



In [ ]:
# 訓練資料格式（跟 DPO 一樣）
{
    "prompt": "問題",
    "chosen": "較好的回答",
    "rejected": "較差的回答"
}

# 訓練目標
loss = -log sigmoid(reward(prompt, chosen) - reward(prompt, rejected))

# 讓 chosen 的分數 > rejected 的分數

### Reward Model 的風險

Reward Model 不是完美的，它會帶來新問題：

**風險 1：Reward Hacking**

In [ ]:
# 問題：Policy 可能學會「欺騙」Reward Model

# 範例：長度偏差
Reward Model 可能無意中學到：「長回答 = 好回答」

結果：
policy 生成超長、重複、廢話連篇的回答來騙高分

# 真實案例
prompt: "1+1=?"
bad_response: "讓我詳細解釋...首先,1是一個自然數...
               [500 words of nonsense]...所以答案是2"
reward: 0.9 (高分！因為很長)

**風險 2：分布外評分不準**

In [ ]:
# Reward Model 在訓練分布外的表現會崩潰

訓練時見過：正常的對話
測試時遇到：極端的、創意的、罕見的回答

結果：亂給分數

**風險 3：過度最佳化**

In [ ]:
# Policy 可能過度適應 RM 的缺陷

# 比喻
就像學生發現老師特別喜歡「舉例說明」，
於是每題都塞一堆例子，但實際上沒回答問題

# LLM 版本
Policy 發現 RM 喜歡「禮貌用語」，
於是每句話都加「非常感謝您的提問」

## PPO 的核心直覺

### 問題：Policy Gradient 很危險

**最簡單的 RL 方法是 Policy Gradient：**

- 如果某個 action 得到高 reward，就增加它的機率
- 如果某個 action 得到低 reward，就降低它的機率
- $$gradient = reward × ∇log π(action|state)$$
- 更新：$$θ_{new} = θ_{old} + α × gradient$$


這個方法 **非常不穩定**，因為：

一次更新可能太大
- policy 突然崩潰
- 生成亂碼
- reward 狂掉
- 下一次更新更糟
- 訓練失敗


### 解決方案：限制更新幅度

```PPO 的核心想法：不要一次改太多```

我們希望：π_new 不要跟 π_old 差太遠



有兩種主流方法：

**方法 1：KL Penalty（DPO 也用這個）**

$$objective = E[reward] - β × KL(π_{new} || π_{old})$$

「追求高 reward，但不要偏離太遠」

**方法 2：Clipped Objective**

# 定義 importance ratio
$$r(θ) = π_θ(a|s) / π_old(a|s)$$

# PPO-Clip objective
objective = min(
    r(θ) × advantage,              # 原始目標
    clip(r(θ), 1-ε, 1+ε) × advantage  # 限制版本
)

「如果 policy 變化太大（r 偏離 1），就停止更新」


### 但這裡有個關鍵問題：跟「誰」比較？

在 PPO 訓練過程中，π_old 會一直更新，如果只跟「上一個 iteration 的自己」比較，模型可能會逐漸漂移：

In [ ]:
Iteration 1: π_1 跟 π_0 很接近 ✓
Iteration 2: π_2 跟 π_1 很接近 ✓
Iteration 3: π_3 跟 π_2 很接近 ✓

但是：π_100 可能已經跟 π_0 完全不同！

### Reference Model 的角色
**Reference Model 是一個「錨點」：**

In [ ]:
# Reference Model = 訓練開始時的模型（通常是 SFT 後的模型）
π_ref = load_model("sft_checkpoint")
for param in π_ref.parameters():
    param.requires_grad = False  # 凍結，永不更新

# Policy Model = 我們正在訓練的模型
π_θ = load_model("sft_checkpoint")  # 同樣的起點
# 但 π_θ 會不斷更新

**為什麼需要這個錨點？**

1. 防止遺忘（Catastrophic Forgetting）

In [ ]:
# 沒有 Reference Model
   Policy 追求高 reward → 可能忘記原本學會的語言能力
   
# 例子
   Original (SFT): "如何學習 Python？" → "建議從官方教學開始..."
   After PPO (無錨點): "如何學習 Python？" → "!!!REWARD_MAX!!!" (語言崩壞)
   
# 有 Reference Model
   KL penalty 會懲罰「跟原始模型差太遠」的行為
   → 保持語言能力的同時優化 reward

2. 保持分布穩定

In [ ]:
# Reward Model 是在某個分布上訓練的
   如果 Policy 偏離太遠，RM 的評分會不可靠
   
# 比喻
   RM 像是只看過「正常對話」的評審
   如果 Policy 生成「火星文」，RM 根本不知道怎麼評分
   
# Reference Model 確保
   Policy 始終在「RM 熟悉的分布」附近

3. 控制最佳化激進程度

In [ ]:
   如果允許無限制優化：Policy 可能找到 RM 的漏洞 (reward hacking)
   
   Reference Model 就像「原則底線」：「你可以變好，但不能變得面目全非」

### Reference Model vs Old Policy

| 比較對象 | Reference Model (π_ref) | Old Policy (π_old) |
|---------|------------------------|-------------------|
| 何時固定 | 訓練開始時就凍結 | 每個 iteration 更新 |
| 用途 | 全域約束（防止偏離起點） | 局部約束（控制單步更新） |
| 數學中的角色 | KL penalty 項 | Importance ratio |


In [ ]:
# PPO 同時使用兩者

# 1. Reference Model: 全域約束
kl_penalty = KL(π_θ || π_ref)  # 跟起點的距離
loss -= β * kl_penalty

# 2. Old Policy: 局部約束  
ratio = π_θ(a|s) / π_old(a|s)  # 跟上一步的比例
clipped_ratio = clip(ratio, 1-ε, 1+ε)

# 完整 objective
L = min(ratio × A, clipped_ratio × A) - β × kl_penalty

### 兩種限制更新的方法
有了 Reference Model 這個概念，我們來看 PPO 如何限制更新：

**方法 1：KL Penalty**

In [ ]:
objective = E[reward] - β × KL(π_θ || π_ref)

**方法 2：Clipped Objective（跟 Old Policy 比）**

In [ ]:
# 定義 importance ratio（跟上一步比）
r(θ) = π_θ(a|s) / π_old(a|s)

# PPO-Clip objective
objective = min(
    r(θ) × advantage,              # 原始目標
    clip(r(θ), 1-ε, 1+ε) × advantage  # 限制版本
)

**實務上 PPO 會同時使用兩種方法：**

In [ ]:
# 雙重保險
L = min(ratio × A, clipped_ratio × A)  # 局部約束 (vs π_old)
    - β × KL(π_θ || π_ref)             # 全域約束 (vs π_ref)

### Advantage

PPO 不直接用 reward，而是用 advantage：

$$advantage = reward - baseline$$

- 不是「這個 action 有多好」（絕對）
- 而是「這個 action 比平均好多少」（相對）

**Baseline 通常用 Value Function V(s)：**


In [ ]:
V(s) = 期望的 total reward

advantage(s, a) = Q(s, a) - V(s)
                = reward + γ V(s') - V(s)  # TD residual

### 總結：PPO 的三層保護機制

1. Clipped Ratio (局部)
   - 限制「這一步」不要變太多
   - 對比：π_θ vs π_old
   
2. KL Penalty (全域)  
   -  限制「累積偏離」不要太遠
   -  對比：π_θ vs π_ref (Reference Model)
   
3. Advantage (相對評估)
   -  用「相對好壞」而非「絕對分數」
   -  避免所有 action 都被強化

## PPO 訓練流程

### 完整流程圖

1. Rollout Phase（採樣階段）：用當前 policy π_θ 生成 N 個 responses
    - prompt_1 → response_1
    - prompt_2 → response_2
    - prompt_N → response_N
2. Reward Phase（評分階段）：用 Reward Model 給每個 response 打分數
    - (prompt_1, response_1) → reward_1
    - (prompt_2, response_2) → reward_2
    - (prompt_N, response_N) → reward_N
3. Advantage 計算階段：用 Value Network 估計 baseline 計算 advantage
    - advantage = reward - V(state)
4. Policy Update（更新階段）：用 PPO objective 更新 policy 、更新 value net 
    - θ_new ← θ_old + gradient
5. 回到步驟 1

### 詳細程式碼流程

In [ ]:
# ===== 初始化 =====
policy_model = load_model("sft_checkpoint")  # 從 SFT 開始
ref_model = load_model("sft_checkpoint")     # 凍結的參考模型
reward_model = load_trained_rm()              # 已訓練好的 RM
value_model = initialize_value_network()      # V(s) 估計器

# 超參數
ppo_epochs = 4          # 每批資料訓練幾輪
clip_epsilon = 0.2      # clip 範圍
beta = 0.01             # KL penalty 係數
batch_size = 256        # 每次採樣多少 prompts

# ===== 主訓練迴圈 =====
for iteration in range(num_iterations):
    
    # ===== Phase 1: Rollout =====
    prompts = sample_prompts(batch_size)  # 從訓練集採樣
    
    with torch.no_grad():
        # 生成 responses
        responses = policy_model.generate(
            prompts,
            max_length=512,
            temperature=1.0,
            do_sample=True
        )
        
        # 計算 old log probs（用於 importance ratio）
        old_logprobs = policy_model.compute_logprobs(
            prompts, responses
        )
        
        # 計算 ref log probs（用於 KL penalty）
        ref_logprobs = ref_model.compute_logprobs(
            prompts, responses
        )
    
    # ===== Phase 2: Reward =====
    with torch.no_grad():
        rewards = reward_model(prompts, responses)  # shape: (batch_size,)
        
        # 計算 KL penalty
        kl_penalty = old_logprobs - ref_logprobs
        rewards = rewards - beta * kl_penalty
    
    # ===== Phase 3: Advantage =====
    with torch.no_grad():
        values = value_model(prompts, responses)  # 估計 V(s)
        advantages = rewards - values             # A = R - V
        
        # 標準化 advantage（穩定訓練）
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
    
    # ===== Phase 4: PPO Update =====
    for ppo_epoch in range(ppo_epochs):
        
        # 重新計算 log probs（因為 policy 已更新）
        new_logprobs = policy_model.compute_logprobs(
            prompts, responses
        )
        
        # 計算 importance ratio
        ratio = torch.exp(new_logprobs - old_logprobs)
        
        # PPO-Clip objective
        surr1 = ratio * advantages
        surr2 = torch.clamp(ratio, 1-clip_epsilon, 1+clip_epsilon) * advantages
        policy_loss = -torch.min(surr1, surr2).mean()
        
        # Value loss（用於訓練 value network）
        value_pred = value_model(prompts, responses)
        value_loss = F.mse_loss(value_pred, rewards)
        
        # 總 loss
        total_loss = policy_loss + 0.5 * value_loss
        
        # 更新
        optimizer.zero_grad()
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(policy_model.parameters(), max_grad_norm=1.0)
        optimizer.step()
    
    # ===== 監控指標 =====
    metrics = {
        "reward_mean": rewards.mean().item(),
        "kl_div": kl_penalty.mean().item(),
        "policy_loss": policy_loss.item(),
        "value_loss": value_loss.item(),
        "ratio_mean": ratio.mean().item(),
        "advantage_mean": advantages.mean().item()
    }
    
    log_metrics(metrics)

### 使用 TRL 函式庫

In [ ]:
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from transformers import AutoTokenizer

# ===== 模型設定 =====

# Policy model（含 value head）
model = AutoModelForCausalLMWithValueHead.from_pretrained("sft_model")

# Reference model（同結構，凍結）
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained("sft_model")
ref_model.eval()

# Reward model（Sequence Classification）
reward_model = AutoModelForSequenceClassification.from_pretrained("reward_model")
reward_model.eval()

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("sft_model")
tokenizer.pad_token = tokenizer.eos_token

# ===== PPO 配置 =====
ppo_config = PPOConfig(
    model_name="sft_model",
    learning_rate=1.4e-5,
    batch_size=256,
    mini_batch_size=64,
    ppo_epochs=4,

    # PPO 核心參數
    cliprange=0.2,
    vf_coef=0.5,
    init_kl_coef=0.01,

    # Adaptive KL
    target_kl=6.0,
    adap_kl_ctrl=True,

    # 穩定性
    cliprange_value=0.2,
    max_grad_norm=1.0,
)

# ===== 初始化 PPO Trainer =====
ppo_trainer = PPOTrainer(
    config=ppo_config,
    model=model,
    ref_model=ref_model,
    tokenizer=tokenizer,
)

# ===== 訓練迴圈 =====
for batch in dataloader:
    prompts = batch["prompt"]

    # Encode prompts
    query_tensors = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
    ).input_ids

    # Rollout（產生回應）
    response_tensors = ppo_trainer.generate(
        query_tensors,
        max_new_tokens=128,
        do_sample=True,
        top_p=0.9,
        temperature=1.0,
    )

    # Decode
    responses = tokenizer.batch_decode(
        response_tensors,
        skip_special_tokens=True,
    )

    # ===== Reward 計算 =====
    # 一般是 prompt + response
    reward_texts = [
        p + r for p, r in zip(prompts, responses)
    ]

    reward_inputs = tokenizer(
        reward_texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
    )

    with torch.no_grad():
        reward_outputs = reward_model(**reward_inputs)
        rewards = reward_outputs.logits.squeeze(-1)

    # ===== PPO Update =====
    stats = ppo_trainer.step(
        query_tensors,
        response_tensors,
        rewards,
    )

    # ===== 監控 =====
    print(f"Mean reward: {stats['ppo/mean_scores']:.2f}")
    print(f"Mean KL: {stats['objective/kl']:.4f}")


## 超參數調優指南

### 推薦起始值

In [ ]:
recommended_config = {
    # 學習率（比 SFT 小 100x）
    "learning_rate": 1e-6,
    
    # PPO 核心
    "clip_epsilon": 0.2,      # 標準值
    "ppo_epochs": 4,          # 不要太多（容易過擬合）
    
    # KL 控制
    "init_kl_coef": 0.02,     # 初始 β
    "target_kl": 6.0,         # 目標 KL
    "adap_kl_ctrl": True,     # 啟用適應性調整
    
    # Batch size
    "batch_size": 256,        # 大 batch 更穩定
    "mini_batch_size": 64,    # 實際更新的 batch
    
    # 價值網路
    "vf_coef": 0.5,          # value loss 的權重
    "cliprange_value": 0.2,   # value clipping
    
    # 穩定性
    "max_grad_norm": 1.0,     # 梯度裁剪
}

### 調參策略

In [ ]:
def tune_hyperparameters():
    """逐步調參的建議流程"""
    
    # Step 1: 先用小資料集測試穩定性
    # 目標：確保 policy 不會崩潰
    
    # Step 2: 固定其他參數，調整學習率
    # 監控：KL 散度（應該 < 10）、reward 趨勢
    
    # Step 3: 調整 KL penalty
    # 如果 KL 太大 → 增加 init_kl_coef
    # 如果 reward 提升太慢 → 減少 init_kl_coef
    
    # Step 4: 調整 clip_epsilon
    # 如果訓練不穩定 → 減小
    # 如果 ratio 很少被 clip → 可增大
    
    # Step 5: 調整 batch size
    # 記憶體允許的情況下，越大越穩定
    
    pass

### 監控指標

In [ ]:
critical_metrics = {
    "reward_mean": "應該單調遞增",
    "kl_div": "應該保持在 < 10",
    "ratio_mean": "應該接近 1.0",
    "ratio_clipped_ratio": "應該 10-30%（太高表示更新太激進）",
    "policy_loss": "應該逐漸下降",
    "value_loss": "應該逐漸下降",
    "explained_variance": "應該 > 0（value network 有學到東西）",
}

## 工程技巧

### 分階段訓練

In [ ]:
# Stage 1: Warmup（謹慎開始）
stage1_config = {
    "lr": 5e-7,
    "clip_epsilon": 0.1,
    "init_kl_coef": 0.05,  # 強約束
    "ppo_epochs": 2,
}
train(model, stage1_config, iterations=500)

# Stage 2: Main Training（正常訓練）
stage2_config = {
    "lr": 1e-6,
    "clip_epsilon": 0.2,
    "init_kl_coef": 0.02,
    "ppo_epochs": 4,
}
train(model, stage2_config, iterations=5000)

# Stage 3: Fine-tuning（謹慎結束）
stage3_config = {
    "lr": 5e-7,
    "clip_epsilon": 0.15,
    "init_kl_coef": 0.03,  # 再次增強約束
    "ppo_epochs": 3,
}
train(model, stage3_config, iterations=500)

### 動態調整 β

In [ ]:
# 適應性 KL 控制（TRL 內建）
if kl_div > target_kl * 1.5:
    # KL 太大，增強懲罰
    beta *= 1.5
    print(f"⬆️ 增加 KL penalty: β = {beta}")
    
elif kl_div < target_kl * 0.5:
    # KL 太小，可以放寬
    beta *= 0.75
    print(f"⬇️ 減少 KL penalty: β = {beta}")

### Reward Scaling

In [ ]:
# 問題：不同 prompt 的 reward 範圍差異很大
# 解決：標準化 reward

def normalize_rewards(rewards):
    """對每個 batch 的 reward 標準化"""
    return (rewards - rewards.mean()) / (rewards.std() + 1e-8)

def whiten_rewards(rewards):
    """更激進的標準化"""
    return (rewards - rewards.mean()) / (rewards.std() + 1e-8)

# 在訓練時使用
rewards_raw = reward_model(prompts, responses)
rewards = normalize_rewards(rewards_raw)

## 常見踩雷點
### 過擬合到訓練分布


問題：

Reward Model 在訓練集上準確率 90%，但遇到新的 response 就亂給分

解決：
1. 用更多樣化的偏好資料訓練 RM
2. 在 RM 訓練中加入 data augmentation
3. 定期用新的 policy 生成樣本，重訓 RM（iterative RLHF）

In [ ]:
# 監控指標
def check_rm_quality(rm, policy):
    """檢查 RM 是否還可靠"""
    
    # 生成新樣本
    new_responses = policy.generate(test_prompts)
    
    # 人工評分一小批
    human_scores = human_annotate(sample(new_responses, 100))
    rm_scores = rm(new_responses)
    
    # 計算相關係數
    correlation = pearson_corr(human_scores, rm_scores)
    
    if correlation < 0.6:
        print("Reward Model 已失效，需要重訓")
        
    return correlation

### 長度偏差

問題：

RM 無意中學到：長度 = 品質

原因：

訓練資料中，好的回答通常比較詳細（= 較長）

結果：

Policy 生成超長、重複的回答來騙高分

In [ ]:
# 方案 1：在 RM 訓練中加入長度控制
reward = rm_score - length_penalty * len(response)

# 方案 2：長度標準化
reward = rm_score / sqrt(len(response))

# 方案 3：用偏好資料中控制長度的樣本
filtered_data = [
    sample for sample in data
    if abs(len(sample['chosen']) - len(sample['rejected'])) < threshold
]

### Policy 崩潰

症狀：

訓練到一半，模型突然開始生成亂碼，reward 暴跌，再也回不來

原因：
1. 學習率太大
2. clip_epsilon 太大
3. KL penalty 太小
4. 某次更新太激進

In [ ]:
# 1. 保守的超參數
lr = 1e-6  # 非常小的學習率
clip_epsilon = 0.1  # 較小的 clip 範圍
init_kl_coef = 0.02  # 較強的 KL 約束

# 2. 梯度裁剪
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)

# 3. 檢查點頻繁儲存
if iteration % 50 == 0:
    save_checkpoint(model, f"iter_{iteration}")

# 4. Early stopping
if kl_div > 10.0:  # KL 過大
    print("KL 散度過大，停止訓練")
    model.load_state_dict(last_stable_checkpoint)
    break

### Reward Hacking

問題：

Policy 找到了 RM 的漏洞，用不該高分的方式騙高分

真實案例：
RM 喜歡「有禮貌」的回答
→ Policy 學會在每句話都加「謝謝」、「請」
→ Reward 很高，但回答品質沒變好

In [ ]:
def detect_reward_hacking():
    """定期人工檢查生成品質"""
    
    samples = policy.generate(test_prompts, n=20)
    
    print("請人工評分這些樣本（1-5分）：")
    for sample in samples:
        print(f"\nPrompt: {sample.prompt}")
        print(f"Response: {sample.response}")
        print(f"RM Score: {sample.rm_score:.2f}")
        
        human_score = input("你的評分: ")
        
        # 如果 RM 高分但人類低分 → reward hacking
        if sample.rm_score > 0.8 and human_score < 3:
            print("可能存在 reward hacking")